# סוכני Azure AI עם תמיכה בפרוטוקול Model Context Protocol (MCP) - Python

מחברת זו מדגימה כיצד להשתמש בסוכני Azure AI עם כלים של פרוטוקול Model Context Protocol (MCP) ב-Python. היא מציגה כיצד ליצור סוכן חכם שיכול לנצל שרתי MCP חיצוניים (כמו Microsoft Learn) לשיפור יכולות באמצעות אימות ללא מפתחות.


## התקנת חבילות Python נדרשות

ראשית, עלינו להתקין את חבילות ה-Python הנחוצות:
- **azure-ai-projects**: SDK מרכזי לפרויקטים של Azure AI  
- **azure-ai-agents**: SDK של Azure AI Agents ליצירה וניהול של סוכנים  
- **azure-identity**: מספק אימות ללא מפתחות באמצעות DefaultAzureCredential  
- **mcp**: יישום של פרוטוקול Model Context עבור Python  


## יתרונות אימות ללא מפתחות

מחברת זו מדגימה **אימות ללא מפתחות**, שמציע מספר יתרונות:
- ✅ **אין צורך לנהל מפתחות API** - משתמש באימות מבוסס זהות של Azure
- ✅ **אבטחה משופרת** - אין סודות שמאוחסנים בקוד או בקבצי תצורה
- ✅ **סבב אישורים אוטומטי** - Azure מטפל בניהול מחזור החיים של האישורים
- ✅ **בקרת גישה מבוססת תפקידים** - משתמש ב-RBAC של Azure להרשאות מדויקות
- ✅ **תמיכה בריבוי סביבות** - עובד בצורה חלקה בין פיתוח לסביבת ייצור

ה-`DefaultAzureCredential` בוחר באופן אוטומטי את מקור האישורים הטוב ביותר הזמין:
1. **Managed Identity** (כאשר פועל ב-Azure)
2. אישורי **Azure CLI** (במהלך פיתוח מקומי)
3. אישורי **Visual Studio**
4. **משתני סביבה** (אם הוגדרו)
5. אימות **דפדפן אינטראקטיבי** (כאופציה אחרונה)


## הגדרת אימות ללא מפתחות

**דרישות מוקדמות לאימות ללא מפתחות:**

### עבור פיתוח מקומי:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### עבור סביבות Azure:
- הפעלת **זהות מנוהלת מוקצת מערכתית** במשאב Azure שלך  
- הקצאת **תפקידי RBAC** מתאימים לזהות המנוהלת:  
  - `Cognitive Services OpenAI User` עבור גישה ל-Azure OpenAI  
  - `AI Developer` עבור גישה לפרויקטים של Azure AI  

### משתני סביבה (אופציונלי):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**אין צורך במפתחות API או במחרוזות חיבור!** 🔐  


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## ייבוא ספריות נדרשות

ייבא את המודולים הנחוצים של Python:
- **os, time**: ספריות סטנדרטיות של Python עבור משתני סביבה ועיכובים
- **AIProjectClient**: לקוח ראשי עבור פרויקטים של Azure AI
- **DefaultAzureCredential**: אימות ללא מפתח עבור שירותי Azure
- **MCP-related classes**: ליצירה וניהול של כלי MCP וטיפול באישורים


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## הגדרת הגדרות שרת MCP

הגדר את תצורת שרת MCP באמצעות משתני סביבה עם ערכי ברירת מחדל:
- **MCP_SERVER_URL**: כתובת ה-URL של שרת MCP (ברירת מחדל היא Microsoft Learn API)
- **MCP_SERVER_LABEL**: תווית לזיהוי שרת MCP (ברירת מחדל היא "mslearn")

גישה זו מאפשרת תצורה גמישה בסביבות שונות.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## יצירת לקוח פרויקט Azure AI (אימות ללא מפתח)

אתחול לקוח פרויקט Azure AI באמצעות **אימות ללא מפתח**:
- **endpoint**: כתובת ה-URL של נקודת הקצה של פרויקט Azure AI Foundry
- **credential**: משתמש ב-`DefaultAzureCredential()` לאימות מאובטח וללא מפתח
- **ללא צורך במפתחות API**: מזהה ומשתמש באופן אוטומטי באישורים הזמינים הטובים ביותר

**תהליך האימות:**
1. בודק זהות מנוהלת (בסביבות Azure)
2. עובר לאישורי Azure CLI (לצורך פיתוח מקומי)
3. משתמש במקורות אישור אחרים לפי הצורך

גישה זו מבטלת את הצורך לנהל מפתחות API או מחרוזות חיבור בקוד שלך.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## יצירת הגדרת כלי MCP

צור כלי MCP שמתחבר לשרת MCP של Microsoft Learn:
- **server_label**: מזהה עבור שרת ה-MCP
- **server_url**: נקודת קצה URL של שרת ה-MCP
- **allowed_tools**: רשימה אופציונלית להגבלת הכלים שניתן להשתמש בהם (רשימה ריקה מאפשרת את כל הכלים)

כלי זה יאפשר לסוכן גישה לתיעוד ולמשאבים של Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## יצירת סוכן וביצוע שיחה (תהליך ללא מפתחות)

החלק המקיף הזה מציג את תהליך העבודה המלא של **סוכן ללא מפתחות**:

1. **יצירת סוכן AI**: הגדרת סוכן עם מודל GPT-4.1 nano וכלי MCP  
2. **יצירת שרשור**: הקמת שרשור שיחה לצורך תקשורת  
3. **שליחת הודעה**: שאל את הסוכן על ההבדלים בין Azure OpenAI ל-OpenAI  
4. **אישור כלי**: אישור אוטומטי של קריאות לכלי MCP כאשר נדרש  
5. **מעקב אחר ביצוע**: מעקב אחר התקדמות הסוכן וטיפול בפעולות נדרשות  
6. **הצגת תוצאות**: הצגת פרטי השיחה והשימוש בכלים  

**תכונות ללא מפתחות:**
- ✅ **ללא סודות מקודדים מראש** - כל האימות מתבצע באמצעות זהות Azure  
- ✅ **מאובטח כברירת מחדל** - משתמש בבקרת גישה מבוססת תפקידים  
- ✅ **פריסה פשוטה** - אין צורך בניהול אישורים  
- ✅ **ידידותי לביקורת** - כל הגישה מתועדת דרך זהות Azure  

הסוכן ישתמש בכלי MCP כדי לגשת למשאבי Microsoft Learn עם אבטחה מלאה וללא צורך בניהול מפתחות API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור הסמכותי. למידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי בני אדם. איננו נושאים באחריות לאי-הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
